In [1]:
!pip install xgboost

In [20]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [3]:
cd ./drive/MyDrive/미니_프로젝트/data

/content/drive/MyDrive/미니_프로젝트/data


# 모든 feature가 다 포함

In [4]:
train_fast = pd.read_csv('./train_fast_all.csv')
test_fast = pd.read_csv('./test_fast_all.csv')
train_slow = pd.read_csv('./train_slow_all.csv')
test_slow = pd.read_csv('./test_slow_all.csv')

In [5]:
train_fast.shape, test_fast.shape, train_slow.shape, test_slow.shape

((102316, 68), (25575, 68), (102294, 68), (25575, 68))

## 급속 충전기

In [6]:
X_fast_train = train_fast.drop(['fast_exist'], axis=1)
y_fast_train = train_fast.fast_exist

X_fast_test = test_fast.drop(['fast_exist'], axis=1)
y_fast_test = test_fast.fast_exist

### XGBoost

In [8]:
XGB = XGBClassifier(random_state=42, max_depth=2)

In [9]:
XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(max_depth=2, random_state=42)

In [11]:
list(X_fast_train.columns)

Index(['학교_scaled', '편의시설_scaled', '통신시설_scaled', '청소년관련시설_scaled',
       '창고_scaled', '차량수_scaled', '주차장_scaled', '주거지_scaled', '전기차_scaled',
       '장례식장_scaled', '자동차관련시설_scaled', '인구_scaled', '의료시설_scaled',
       '위험물저장시설_scaled', '위락시설_scaled', '운수시설_scaled', '완속_scaled',
       '업무시설_scaled', '숙박시설_scaled', '상하수도및가스시설_scaled', '산업단지거리_scaled',
       '발전시설_scaled', '문화및집회시설_scaled', '묘지관련시설_scaled', '면적_scaled',
       '님비시설_scaled', '농축산업시설_scaled', '녹지_scaled', '노유자시설_scaled',
       '교육연구및복지시설_scaled', '광장_scaled', '관광휴게시설_scaled', '공장_scaled',
       '공원_scaled', '공공시설_scaled', '고도_scaled', '계획부지_scaled',
       '개발제한구역비율_scaled', '개발부지_scaled', 'width_5_scaled', 'up_lanes_6_scaled',
       'up_lanes_5_scaled', 'taxiStation_scaled', 'dw_lanes_5_scaled',
       'busStation', 'dw_lanes_0', 'dw_lanes_1', 'dw_lanes_2', 'dw_lanes_3',
       'dw_lanes_4', 'up_lanes_1', 'up_lanes_2', 'up_lanes_3', 'up_lanes_4',
       'width_1', 'width_2', 'width_3', 'width_4', '급속', '버스', '승용차', 

In [16]:
feature_imp = {i:j for i,j in zip(list(X_fast_train.columns), list(XGB.feature_importances_))}

In [18]:
sorted_dict = sorted(feature_imp.items(), key = lambda item: item[1])
sorted_dict

[('편의시설_scaled', 0.0),
 ('통신시설_scaled', 0.0),
 ('장례식장_scaled', 0.0),
 ('의료시설_scaled', 0.0),
 ('운수시설_scaled', 0.0),
 ('상하수도및가스시설_scaled', 0.0),
 ('발전시설_scaled', 0.0),
 ('문화및집회시설_scaled', 0.0),
 ('묘지관련시설_scaled', 0.0),
 ('녹지_scaled', 0.0),
 ('광장_scaled', 0.0),
 ('공원_scaled', 0.0),
 ('width_5_scaled', 0.0),
 ('up_lanes_6_scaled', 0.0),
 ('up_lanes_5_scaled', 0.0),
 ('taxiStation_scaled', 0.0),
 ('dw_lanes_5_scaled', 0.0),
 ('dw_lanes_0', 0.0),
 ('dw_lanes_3', 0.0),
 ('dw_lanes_4', 0.0),
 ('up_lanes_1', 0.0),
 ('up_lanes_2', 0.0),
 ('width_2', 0.0),
 ('width_3', 0.0),
 ('width_4', 0.0),
 ('승용차', 0.0),
 ('유원지', 0.0),
 ('혼잡빈도강도', 0.0),
 ('화물차', 0.0017297387),
 ('dw_lanes_1', 0.0022048499),
 ('주차장_scaled', 0.0028906185),
 ('width_1', 0.0029936626),
 ('계획부지_scaled', 0.0031819677),
 ('공공시설_scaled', 0.0037130078),
 ('님비시설_scaled', 0.003950299),
 ('위험물저장시설_scaled', 0.0039546667),
 ('busStation', 0.0039742426),
 ('노유자시설_scaled', 0.0050258203),
 ('전기차_scaled', 0.0051640854),
 ('인구_scaled', 0.005425

In [26]:
XGB_fs = SelectFromModel(XGB)

In [27]:
XGB_fs.fit(X_fast_train, y_fast_train)
new_X = XGB_fs.transform(X_fast_train)

In [37]:
XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(max_depth=2, random_state=42)

In [30]:
new_X_test = XGB_fs.transform(X_fast_test)

In [38]:
y_fast_prob = XGB.predict_proba(X_fast_test)

In [39]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.07631160572337044
0.2 :0.0935672514619883
0.3 :0.11358024691358026
0.4 :0.14024390243902438
0.5 :0.17829457364341086
0.6 :0.20388349514563103
0.7 :0.2285714285714286
0.8 :0.28776978417266186
0.9 :0.36


* **오버피팅!!!!!!!!!**

In [34]:
y_fast_pred_train = XGB.predict(new_X)
f1_score(y_fast_train, y_fast_pred_train)

0.9934940086617079

## 완속 충전기

In [ ]:
X_slow_train = train_slow.drop(['slow_exist'], axis=1)
y_slow_train = train_slow.slow_exist

X_slow_test = test_slow.drop(['slow_exist'], axis=1)
y_slow_test = test_slow.slow_exist

### XGBoost

In [ ]:
XGB = XGBClassifier(random_state=42, max_depth=2)

In [ ]:
XGB.fit(X_slow_train, y_slow_train)

XGBClassifier(max_depth=2, random_state=42)

In [ ]:
y_slow_prob = XGB.predict_proba(X_slow_test)

In [ ]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

0.1 :0.0
0.2 :0.0
0.3 :0.0
0.4 :0.0
0.5 :0.0
0.6 :0.0
0.7 :0.0
0.8 :0.0
0.9 :0.0


In [ ]:
y_slow_pred_train = XGB.predict(X_slow_train)
f1_score(y_slow_train, y_slow_pred_train)

0.9930683649496146